In [20]:
# Standard libs + requests + dotenv for config and HTTP
# Load .env from repo root to populate WHOOP_* vars
# WHOOP_* identifiers are required for OAuth
# WHOOP API endpoints and version selector
# Load environment variables and API configuration

# Import necessary modules for environment handling, HTTP requests, file operations, and JSON processing
import os, json, time, pathlib, urllib.parse
import requests
from dotenv import load_dotenv, find_dotenv

# Locate and load the .env file to access environment variables
env_path = find_dotenv(usecwd=True)
print("Found .env at:", env_path)
load_dotenv(env_path, override=True)

# Retrieve Whoop API credentials and redirect URI from environment variables
WHOOP_CLIENT_ID = os.getenv("WHOOP_CLIENT_ID")
WHOOP_CLIENT_SECRET = os.getenv("WHOOP_CLIENT_SECRET")
WHOOP_REDIRECT_URI = os.getenv("WHOOP_REDIRECT_URI", "https://oauth.pstmn.io/v1/callback")

# Ensure required credentials are present
assert WHOOP_CLIENT_ID, "Missing WHOOP_CLIENT_ID"
assert WHOOP_CLIENT_SECRET, "Missing WHOOP_CLIENT_SECRET"

# Define URLs for Whoop API token exchange and base API endpoint
TOKEN_URL = "https://api.prod.whoop.com/oauth/oauth2/token"
API_BASE = "https://api.prod.whoop.com/developer"
#API_VERSION = "v2"

# Print the redirect URI for confirmation
print("WHOOP_REDIRECT_URI =", WHOOP_REDIRECT_URI)



Found .env at: /home/grifin/projects/whoop-ai-fitness-lab/.env
WHOOP_REDIRECT_URI = https://oauth.pstmn.io/v1/callback


In [21]:
# Cell 2
# Token cache location on disk
# In-memory token_state stores current access/refresh tokens
# Save/restore helpers to persist tokens between runs
# Token persistence helpers (load/save to disk)

token_file = pathlib.Path("data/secrets/whoop_tokens.json")
token_file.parent.mkdir(parents=True, exist_ok=True)

token_state = None

def save_tokens():
    if not token_state or "access_token" not in token_state:
        raise ValueError("token_state missing access_token; cannot save tokens")
    token_file.write_text(json.dumps({
        "access_token": token_state["access_token"],
        "refresh_token": token_state.get("refresh_token"),
        "expires_at": token_state["expires_at"],
        "saved_at": time.time()
    }, indent=2))
    print(f"Saved tokens to: {token_file}")

def load_tokens():
    global token_state
    if token_file.exists():
        saved = json.loads(token_file.read_text())
        token_state = {
            "access_token": saved["access_token"],
            "refresh_token": saved.get("refresh_token"),
            "expires_at": saved.get("expires_at", 0),
        }
        print("Loaded saved tokens.")
    else:
        print("No saved tokens yet.")


In [23]:
# Cell 3
# Exchange auth code for access/refresh token pair
# Refresh access token when expired using refresh_token
# OAuth token exchange and refresh helpers

def exchange_code_for_tokens(auth_code: str):
    global token_state
    data = {
        "grant_type": "authorization_code",
        "client_id": WHOOP_CLIENT_ID,
        "client_secret": WHOOP_CLIENT_SECRET,
        "redirect_uri": WHOOP_REDIRECT_URI,
        "code": auth_code.strip(),
    }
    r = requests.post(TOKEN_URL, data=data, timeout=30)
    r.raise_for_status()
    j = r.json()

    existing_refresh = token_state.get("refresh_token") if token_state else None
    new_refresh = j.get("refresh_token") or existing_refresh

    token_state = {
        "access_token": j["access_token"],
        "refresh_token": new_refresh,
        "expires_at": time.time() + int(j.get("expires_in", 3600)) - 30
    }
    save_tokens()
    if token_state.get("refresh_token"):
        print("Exchanged code and saved tokens.")
    else:
        print("Exchanged code and saved access token (no refresh token returned).")

def refresh_if_needed():
    global token_state
    if not token_state:
        return
    if time.time() < token_state.get("expires_at", 0):
        return

    refresh_token = token_state.get("refresh_token")
    if not refresh_token:
        print("Access token expired and no refresh token is available. Reauthorize to continue.")
        token_state = None
        return

    data = {
        "grant_type": "refresh_token",
        "client_id": WHOOP_CLIENT_ID,
        "client_secret": WHOOP_CLIENT_SECRET,
        "refresh_token": refresh_token,
        "scope": "offline",
    }
    r = requests.post(TOKEN_URL, data=data, timeout=30)
    if r.status_code >= 400:
        try:
            err = r.json()
        except ValueError:
            err = r.text
        print("Refresh failed:", err)
        if isinstance(err, dict) and err.get("error") == "invalid_grant":
            print("Refresh token invalid. Reauthorize to continue.")
            token_state = None
        r.raise_for_status()

    j = r.json()
    token_state = {
        "access_token": j["access_token"],
        "refresh_token": j.get("refresh_token") or refresh_token,
        "expires_at": time.time() + int(j.get("expires_in", 3600)) - 30
    }
    save_tokens()
    print("Refreshed access token.")




In [24]:
# Cell 4
# Load saved tokens if present
# If missing, exchange an auth code for tokens
# Otherwise refresh to ensure validity
# Ensure a valid access token is available

load_tokens()

if token_state is None:
    AUTH_CODE = "fUGEAIREYGRoIqq_WSMJOOu81J6PDsHOnlWs4pX_8qw.0z-T0kkWevsNGKAish7UBM2gaX5L6K5fi1BL-1gOffI"
    exchange_code_for_tokens(AUTH_CODE)

else:
    refresh_if_needed()
    print("Token ready.")



Loaded saved tokens.
Saved tokens to: data/secrets/whoop_tokens.json
Refreshed access token.
Token ready.


In [25]:
# Cell 5
# Map v2-friendly aliases to actual endpoint paths
# Normalize paths so callers can keep v1-style names
# GET wrapper with auth + error logging
# Pagination helper to collect all records
# API request helpers with v2 path normalization

def whoop_get(path, params=None):
    refresh_if_needed()
    headers = {"Authorization": f"Bearer {token_state['access_token']}"}
    url = f"{API_BASE.rstrip('/')}/{path.lstrip('/')}"
    resp = requests.get(url, headers=headers, params=params, timeout=30)
    if resp.status_code >= 400:
        print("url:", resp.url)
        print("status:", resp.status_code)
        print("body:", resp.text[:500])
    resp.raise_for_status()
    return resp.json()


def whoop_get_all(path, start, end, limit=25):
    all_records = []
    next_token = None

    while True:
        params = {"start": start, "end": end, "limit": limit}
        if next_token:
            params["nextToken"] = next_token

        page = whoop_get(path, params=params)
        all_records.extend(page.get("records", []))
        next_token = page.get("nextToken")
        if not next_token:
            break

    return all_records


In [26]:
# Cell 6
# Define a 90-day UTC window
# Fetch records across endpoints using shared helper
# Quick sanity check: tuple of record counts
# Pull recent recovery, sleep, workout, and cycle records

from datetime import datetime, timedelta, timezone

end_dt = datetime.now(timezone.utc).replace(microsecond=0)
start_dt = end_dt - timedelta(days=90)

start = start_dt.isoformat().replace("+00:00", "Z")
end = end_dt.isoformat().replace("+00:00", "Z")

recovery_records = whoop_get_all("/v2/recovery", start, end)
sleep_records    = whoop_get_all("/v2/activity/sleep", start, end)
workout_records  = whoop_get_all("/v2/activity/workout", start, end)
cycle_records    = whoop_get_all("/v2/cycle", start, end)


len(recovery_records), len(sleep_records), len(workout_records), len(cycle_records)



(25, 25, 25, 25)

In [27]:
# Cell 7
# Create output directory for raw exports
# Timestamped filenames keep runs distinct
# Write each record list to JSON
# Persist raw records to timestamped JSON files

out_dir = pathlib.Path("data/raw")
out_dir.mkdir(parents=True, exist_ok=True)

stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
(out_dir / f"recovery_{stamp}.json").write_text(json.dumps(recovery_records, indent=2))
(out_dir / f"sleep_{stamp}.json").write_text(json.dumps(sleep_records, indent=2))
(out_dir / f"workout_{stamp}.json").write_text(json.dumps(workout_records, indent=2))
(out_dir / f"cycle_{stamp}.json").write_text(json.dumps(cycle_records, indent=2))

print("Saved raw record lists to:", out_dir.resolve())





Saved raw record lists to: /home/grifin/projects/whoop-ai-fitness-lab/.venv/data/raw
